In [1]:
from io import BytesIO
import  torch_geometric
from urllib.request import urlopen
from zipfile import ZipFile
from torch_geometric.data import HeteroData
import torch_geometric.transforms as T
import dill as pickle
import torch
import pandas as pd
import numpy as np
from zipfile import ZipFile

### Zip Graph Model

In [ ]:
with ZipFile('final_book_model.zip', 'r') as f:
    f.extractall("model")
#extract in different directory

### Download ratings, users and books

In [ ]:
url = 'http://www2.informatik.uni-freiburg.de/~cziegler/BX/BX-CSV-Dump.zip'
with urlopen(url) as zurl:
    with ZipFile(BytesIO(zurl.read())) as zfile:
        zfile.extractall('.')

### Graph Data

In [2]:
ratings = pd.read_csv('BX-Book-Ratings.csv', sep=';', encoding='latin-1')
users = pd.read_csv('BX-Users.csv', sep=';', encoding='latin-1')
books = pd.read_csv('BX-Books.csv', sep=';', encoding='latin-1', error_bad_lines=False)

C:\Users\morit\AppData\Local\Temp\ipykernel_27536\1575487172.py:3: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  books = pd.read_csv('BX-Books.csv', sep=';', encoding='latin-1', error_bad_lines=False)
Skipping line 6452: expected 8 fields, saw 9
Skipping line 43667: expected 8 fields, saw 10
Skipping line 51751: expected 8 fields, saw 9

Skipping line 92038: expected 8 fields, saw 9
Skipping line 104319: expected 8 fields, saw 9
Skipping line 121768: expected 8 fields, saw 9

Skipping line 144058: expected 8 fields, saw 9
Skipping line 150789: expected 8 fields, saw 9
Skipping line 157128: expected 8 fields, saw 9
Skipping line 180189: expected 8 fields, saw 9
Skipping line 185738: expected 8 fields, saw 9

Skipping line 209388: expected 8 fields, saw 9
Skipping line 220626: expected 8 fields, saw 9
Skipping line 227933: expected 8 fields, saw 11
Skipping line 228957: expected 8 fields, saw 1

In [3]:
ratings_filtered = ratings.loc[ratings["Book-Rating"] >= 8]
ratings_filtered = ratings_filtered.loc[ratings_filtered['ISBN'].isin(books['ISBN'].unique()) & ratings_filtered['User-ID'].isin(users['User-ID'].unique())]
mapping_user = { user_id: index for index, user_id in enumerate(ratings_filtered["User-ID"].unique())}
mapping_item = { isbn_id: index for index, isbn_id in enumerate(ratings_filtered["ISBN"].unique())}

df_mapping_user = pd.DataFrame()
df_mapping_user["user_id"] = mapping_user.keys()
df_mapping_user["user_id_mapped"] = mapping_user.values()

df_mapping_item = pd.DataFrame()
df_mapping_item["isbn_id"] = mapping_item.keys()
df_mapping_item["isbn_id_mapped"] = mapping_item.values()

In [4]:
ratings_filtered_m = ratings_filtered.merge(df_mapping_user, left_on = "User-ID", right_on="user_id", how = "left")
ratings_filtered_m = ratings_filtered_m.merge(df_mapping_item, left_on = "ISBN", right_on="isbn_id", how = "left")


In [5]:

edge_index_user_to_isbn = ratings_filtered_m[["user_id_mapped", "isbn_id_mapped"]]
edge_index_user_to_isbn_user = torch.from_numpy(edge_index_user_to_isbn.user_id_mapped.values)
edge_index_user_to_isbn_isbn =torch.from_numpy( edge_index_user_to_isbn.isbn_id_mapped.values)
edge_index_user_to_isbn_user
edge_index_user_to_isbn_isbn

edge_index_user_to_isbn = torch.stack([edge_index_user_to_isbn_user, edge_index_user_to_isbn_isbn], dim=0)

book_x = books[["ISBN", "Year-Of-Publication", "Publisher"]]
book_x = book_x[(book_x["ISBN"].isin(list(ratings_filtered_m.isbn_id.unique())) )]

book_x = book_x.merge(df_mapping_item,left_on =  "ISBN",right_on = "isbn_id", how= "left")
book_x = book_x.sort_values(by=['isbn_id_mapped'])

book_x = book_x.set_index('isbn_id_mapped')
book_x = book_x[["Year-Of-Publication", "Publisher"]]


for c in [ "Year-Of-Publication", "Publisher"]:
    
    print(f'{c} {book_x[c].nunique()}')
    
book_x = pd.get_dummies(book_x, columns=["Publisher"], prefix=["publisher"])

book_x["Year-Of-Publication"]= book_x["Year-Of-Publication"].astype(int)



print(book_x.info())


torch_isbn = torch.tensor(book_x.values)
torch_isbn.shape

Year-Of-Publication 166
Publisher 8750
<class 'pandas.core.frame.DataFrame'>
Int64Index: 98417 entries, 0 to 98416
Columns: 8751 entries, Year-Of-Publication to publisher_Ã?lbaum
dtypes: int32(1), uint8(8750)
memory usage: 822.4 MB
None


torch.Size([98417, 8751])

In [6]:

data = HeteroData()

data["user"].node_id = torch.from_numpy(ratings_filtered_m.user_id_mapped.unique())
data["isbn"].node_id =torch.from_numpy(ratings_filtered_m.isbn_id_mapped.unique())
data["isbn"].x = torch_isbn
data["user", "review", "isbn"].edge_index  = edge_index_user_to_isbn
data = T.ToUndirected()(data)

a = data.to_dict()

with open('data_graph.pkl', 'wb') as fp:
    pickle.dump(a, fp)
    print('dictionary saved successfully to file')

dictionary saved successfully to file
